# Chapter 9: Getting Data

In [ ]:
from __future__ import division
from bokeh.io import output_notebook
from bs4 import BeautifulSoup
import re, requests
from time import sleep

In [ ]:
output_notebook()

## Web Scraping

In [ ]:
def is_video(td):
    """it's a video if it has exactly one pricelabel, and if
    the stripped text inside that pricelabel starts with 'Video'"""
    pricelabels = td('span', 'pricelabel')
    return (len(pricelabels) == 1 and pricelabels[0].text.strip().startswith("Video"))

In [ ]:
def book_info(td):
    title = td.find("div", "thumbheader").a.text
    author_name = td.find("div","AuthorName").text
    authors = [x.strip() for x in re.sub("^By ", "", author_name).split(',')]
    isbn_link = td.find("div","thumbheader").a.get("href")
    isbn = re.match("/product/(.*)\.do", isbn_link).group(1)
    date = td.find("span", "directorydate").text.strip()
    return {
        "title"   : title,
        "authors" : authors,
        "isbn"    : isbn, 
        "date"    : date
    }

In [ ]:
base_url = "http://shop.oreilly.com/category/browse-subjects/data.do?sortby=publicationDate&page="
books = []
NUM_PAGES = 38
for page_num in range(1, NUM_PAGES+1):
    print "souping page", page_num, ",", len(books), " found so far"
    url = base_url + str(page_num)
    soup = BeautifulSoup(requests.get(url).text, 'html5lib')
    
    for td in soup('td', 'thumbtext'):
        if not is_video(td):
            books.append(book_info(td))
    sleep(30)